In [ ]:
!pip install transformers evaluate peft

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import CLIPImageProcessor, GPT2Tokenizer
import torch.nn as nn
from transformers import CLIPProcessor, CLIPModel, GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoTokenizer
from typing import Tuple
import torch.optim as optim
from torch.nn import functional as nnf
from tqdm import tqdm
from sklearn.metrics import accuracy_score,roc_auc_score
from nltk.translate.bleu_score import sentence_bleu
import pdb
from evaluate import load
import collections
from torch.cuda.amp import autocast
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from peft import (
    get_peft_model,
    LoraConfig,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
    TaskType,
)

## VQA-RAD-Dataset

In [4]:
class VQARADDataset(Dataset):
    def __init__(self, csv_file, img_dir, train_setting=True): #  transform=None
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.vqa_rad_frame = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.preprocess = CLIPImageProcessor.from_pretrained('flaviagiammarino/pubmed-clip-vit-base-patch32')
        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.train_setting = train_setting

    def __len__(self):
        return len(self.vqa_rad_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.vqa_rad_frame.iloc[idx, 0])
        image = Image.open(img_name)
        image = self.preprocess(image, return_tensors="pt")
        questions = self.vqa_rad_frame.iloc[idx, 2]
        answers = self.vqa_rad_frame.iloc[idx, 4]
        tokens, mask, q_len  = self.pad_sequences(idx, questions, answers)
        tokens = tokens.long()
        mask = mask.long()
        q_len = q_len
        sample = {'image': image['pixel_values'].squeeze(0), 'tokens': tokens, 'mask': mask, 'q_len' : q_len, 'answers' : answers, 'questions' : questions}

        return sample

    def pad_sequences(self,index, questions, answers):
        m = [torch.tensor(self.gpt2_tokenizer.encode('question: ')),torch.tensor(self.gpt2_tokenizer.encode(' context:')),torch.tensor(self.gpt2_tokenizer.encode('answer ')),torch.tensor(self.gpt2_tokenizer.encode('<|endoftext|>'))]
        m_mask = [torch.ones(len(self.gpt2_tokenizer.encode('question: '))),torch.ones(len(self.gpt2_tokenizer.encode(' context:'))),torch.ones(len(self.gpt2_tokenizer.encode('answer '))),torch.zeros(len(self.gpt2_tokenizer.encode('<|endoftext|>')))]

        if self.train_setting:
            q=torch.tensor(self.gpt2_tokenizer.encode(str(questions)))
            a=torch.tensor(self.gpt2_tokenizer.encode(str(answers)))
            q,q_mask,leftover_tokens = self.make_padding(16,q,question=True)
            q_len = m[0].size(0) + q.size(0) + m[1].size(0)
            a,a_mask,_ = self.make_padding(6 ,a,leftover_tokens=leftover_tokens)
            if len((a==0).nonzero())!=0:
                pad_start = (a==0).nonzero()[0]
            else:
                pad_start=[]
            a = torch.cat((a,m[3])) if len(pad_start)==0 else torch.cat((a[:pad_start],m[3],a[pad_start:]))
            q = torch.cat((m[0],q,m[1],torch.ones(1),m[2],a))

            q_mask = torch.cat((m_mask[0],q_mask,m_mask[1],torch.ones(1),m_mask[2],a_mask,m_mask[3]))

            return q,q_mask, q_len
        else:
            q = torch.tensor(self.gpt2_tokenizer.encode(str(questions)))
            q,q_mask,_ = self.make_padding_test_setting(24,q)
            q_len = m[0].size(0) + q.size(0) + m[1].size(0)
            q = torch.cat((m[0],q,m[1],torch.ones(1),m[2]))

            q_mask = torch.cat((m_mask[0],q_mask,m_mask[1],torch.ones(1),m_mask[2]))
            return q,q_mask,q_len


    def make_padding(self, max_len, tokens, question=False,leftover_tokens=0):
        padding = max_len - tokens.size(0)
        if padding > 0:
            if question:
                leftover_tokens = padding
                mask = torch.ones(tokens.size(0))
            else:
                tokens = torch.cat((tokens, torch.zeros(padding+leftover_tokens)))
                mask = torch.zeros(max_len+leftover_tokens)

        elif padding==0:
            if question:
                mask = torch.ones(tokens.size(0))
            else:
                mask = torch.zeros(tokens.size(0)+leftover_tokens)
                tokens = torch.cat((tokens,torch.zeros(leftover_tokens)))


        elif padding < 0:
            if question:
                tokens = tokens[:max_len]
                mask = torch.ones(max_len)
            else:
                tokens = torch.cat((tokens[:max_len], torch.zeros(leftover_tokens)))
                mask = torch.zeros(max_len+ leftover_tokens)
        return tokens, mask, leftover_tokens

    def make_padding_test_setting(self, max_len, tokens,do_padding=False):
        padding = max_len - tokens.size(0)
        padding_len = 0
        if padding > 0:
            if do_padding:
                mask = torch.cat((torch.ones(tokens.size(0)),torch.zeros(padding)))
                tokens = torch.cat((tokens,torch.zeros(padding)))
                padding_len = padding
            else:
                mask = torch.ones(tokens.size(0))
        elif padding ==0:
            mask = torch.ones(max_len)
        elif padding < 0:
            tokens = tokens[:max_len]
            mask = torch.ones(max_len)
        return tokens, mask, padding_len


def get_loaders(csv_file='/content/drive/MyDrive/Medical_VQA/data/vqa_rad.csv', img_dir='/content/drive/MyDrive/data/img', batch_size=32, split_ratio=(0.8, 0.1, 0.1)):
    """
    Returns training, validation, and test data loaders.
    Args:
        csv_file (string): Path to the csv file.
        img_dir (string): Directory with all the images.
        batch_size (int): Batch size for DataLoader.
        transform (callable, optional): Optional transform to be applied on image.
        split_ratio (tuple): Ratios for train, val, and test split. They should sum to 1.
    """

    assert sum(split_ratio) == 1, "Split ratios should sum to 1."

    train_dataset = VQARADDataset(csv_file='/content/drive/MyDrive/data/vqa_rad_train.csv', img_dir=img_dir, train_setting = True)
    val_dataset = VQARADDataset(csv_file='/content/drive/MyDrive/data/vqa_rad_valid.csv', img_dir=img_dir, train_setting = False)
    test_dataset = VQARADDataset(csv_file='/content/drive/MyDrive/data/vqa_rad_test.csv', img_dir=img_dir, train_setting = False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    return train_loader, val_loader, test_loader

## Mapping

In [5]:
class MLP(nn.Module):
    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(nn.Dropout(p=0.3))
                layers.append(act())
        self.model = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

## VQA_Model

In [6]:
def get_peft_config(mode):

    if mode == "lora":
      peft_config = LoraConfig(
          task_type=TaskType.CAUSAL_LM, inference_mode=False,
          r=8,
          lora_alpha=32, lora_dropout=0.1
        )
    elif mode == "prefix":
        peft_config = PrefixTuningConfig(
            task_type=TaskType.CAUSAL_LM,
            num_virtual_tokens=32,
            encoder_hidden_size=1024,
            prefix_projection=True,
        )
    elif mode == "ptuning":
        peft_config = PromptEncoderConfig(
            task_type=TaskType.CAUSAL_LM,
            num_virtual_tokens=32,
            encoder_hidden_size=1024,
        )
    elif mode == "prompt":
        peft_config = PromptTuningConfig(
            task_type=TaskType.CAUSAL_LM,
            num_virtual_tokens=32,
        )

    return peft_config

In [7]:
class VQAModel(nn.Module):
    def __init__(self):
        super(VQAModel, self).__init__()

        self.clip_model = CLIPModel.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")
        self.clip_processor = CLIPProcessor.from_pretrained("flaviagiammarino/pubmed-clip-vit-base-patch32")
        self.gpt2_model = GPT2LMHeadModel.from_pretrained("cemilcelik/distilgpt2_pubmed")
        # self.mode = "lora" # ["lora", "prefix", "ptuning", "prompt"]
        # self.peft_config = get_peft_config(self.mode)
        # self.gpt2_model = get_peft_model(self.gpt2_model, self.peft_config)
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.mapper = MLP(sizes=(512, 768, 768))

    def forward(self, images, tokens, mask, q_len):
        image_features = self.clip_model.get_image_features(images)
        image_features = self.mapper(image_features).view(-1, 1, 768)
        embedding = self.gpt2_model.transformer.wte(tokens)

        for b in range(embedding.shape[0]):
            # insert the visual prefix after the question
            embedding[b,q_len[b]:q_len[b]+1,:] = image_features[b]

        return self.gpt2_model(inputs_embeds=embedding, attention_mask=mask).logits

    def generate(self, images, tokens, mask, q_len):
        image_features = self.clip_model.get_image_features(images)
        image_features = self.mapper(image_features).view(-1, 1, 768)
        embedding_txt = self.gpt2_model.transformer.wte(tokens)
        embedding_txt[q_len:q_len+1,:] = image_features
        return embedding_txt

##Training

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 5e-3
MODEL_PATH = '/content/drive/MyDrive/data/best_model.pth'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, val_loader, test_loader = get_loaders(batch_size=BATCH_SIZE)
model = VQAModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

def train(model, dataloader, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch in tqdm(train_loader, desc="Training"):
        images, tokens, mask, q_len = batch['image'], batch['tokens'], batch['mask'], batch['q_len']
        images = images.to(device)
        tokens = tokens.to(device)
        mask = mask.to(device)
        q_len = q_len.to(device)
        optimizer.zero_grad()
        logits = model(images, tokens, mask, q_len)

        # shift = 10 if args.setting=="p_tuning" or args.setting=="prompttuning" else 0
        shift = 0
        loss = 0
        for b in range(logits.size(0)):
                        condensed_tokens = tokens[b,q_len[b]+1+1:]
                        condensed_logits = logits[b,shift+ q_len[b]+1:-1]

                        loss += nnf.cross_entropy(condensed_logits.reshape(-1,logits.shape[-1]), condensed_tokens.flatten(), ignore_index=0)

        loss=loss/logits.size(0)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(dataloader)
    return avg_loss

def validate(model, dataloader, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validating"):
          images, tokens, mask, q_len = batch['image'], batch['tokens'], batch['mask'], batch['q_len']
          images = images.to(device)
          tokens = tokens.to(device)
          mask = mask.to(device)
          q_len = q_len.to(device)
          optimizer.zero_grad()
          logits = model(images, tokens, mask, q_len)
          # shift = 10 if args.setting=="p_tuning" or args.setting=="prompttuning" else 0
          shift = 0
          for b in range(logits.size(0)):
                          condensed_tokens = tokens[b,q_len[b]+1+1:]
                          condensed_logits = logits[b,shift+ q_len[b]+1:-1]

                          loss += nnf.cross_entropy(condensed_logits.reshape(-1,logits.shape[-1]), condensed_tokens.flatten(), ignore_index=0)

          loss=loss/logits.size(0)
          running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    return avg_loss


best_val_loss = float('inf')
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    train_loss = train(model, train_loader, optimizer, device)
    print(f"Train loss: {train_loss:.4f}")

    # val_loss= validate(model, val_loader, device)
    # print(f"Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}")
    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss

torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

print("Training complete.")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 7da93d0d-fd0d-4e6b-a634-6e0e60d27384)')' thrown while requesting HEAD https://huggingface.co/flaviagiammarino/pubmed-clip-vit-base-patch32/resolve/main/vocab.json


Epoch 1/10


Training: 100%|██████████| 113/113 [03:53<00:00,  2.07s/it]


Train loss: 3.1863
Epoch 2/10


Training: 100%|██████████| 113/113 [01:17<00:00,  1.46it/s]


...

Train loss: 1.5450
Epoch 9/10


Training: 100%|██████████| 113/113 [01:16<00:00,  1.47it/s]


Train loss: 1.5663
Epoch 10/10


Training: 100%|██████████| 113/113 [01:17<00:00,  1.46it/s]


Train loss: 1.5624
Model saved to /content/drive/MyDrive/data/best_model.pth
Training complete.


## Inference and Evaluation

In [9]:
def treebank_tokenize(s):
    return TreebankWordTokenizer().tokenize(s)
def generate_beam(
    model,
    tokenizer,
    beam_size: int = 5,
    generated=None,
    entry_length=65,
    temperature=1.0,
    stop_token: str = "<|endoftext|>",
):
    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        for i in range(entry_length):
            outputs = model.gpt2_model(inputs_embeds=generated)
            logits = outputs.logits

            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

            logits = logits.softmax(-1).log()
            # final_logit

            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(
                    beam_size, -1
                )
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]

            next_token_embed = model.gpt2_model.transformer.wte(next_tokens.squeeze()).view(generated.shape[0], 1, -1)

            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [
        tokenizer.decode(output[: int(length)])
        for output, length in zip(output_list, seq_lengths)
    ]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts

In [10]:
def eval_gpt_open_ended(model, dataloader):
    model.eval()
    model= model.to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    bleu_avg1=0.
    f1_avg = 0.
    acc = 0.
    acc_oe = 0.
    acc_yn = 0.
    c_oe =1e-9
    c_yn =1e-9

    for batch in tqdm(dataloader, desc="Testing"):
      images, tokens, mask, q_len = batch['image'], batch['tokens'], batch['mask'], batch['q_len']
      images = images.to(device)
      tokens = tokens.to(device)
      mask = mask.to(device)
      q_len = q_len.to(device)

      with autocast(dtype=torch.float16):
        with torch.no_grad():
            embed = model.generate(images, tokens, mask, q_len)
            out_text = generate_beam(model, model.tokenizer,generated=embed,entry_length=30, temperature=1)[0]

      out_text = out_text.split("<|endoftext|>")[0]

      print('Question: ', batch['questions'])
      print('Answer: ', batch['answers'])
      print("Generated_Answer: ", out_text)

      if out_text.lower()==(batch['answers'][0]).lower():
        acc+=1
      if (batch['answers'][0]).lower()=='yes' or (batch['answers'][0]).lower()=='no':
        if out_text.lower()==(batch['answers'][0]).lower():
          acc_yn+=1
        c_yn+=1


        reference = [str((batch['answers'][0]).lower())]
        candidate = [out_text]

        bleu_1 = sentence_bleu(reference[0], candidate[0], weights=(1, 0, 0, 0))
        f1_avg += compute_f1(tokenizer.encode(reference[0]),tokenizer.encode(candidate[0]))
        bleu_avg1+=bleu_1

    # print('------------')
    print("BLEU {}".format(round(bleu_avg1/len(dataloader),3)))
    print("F1 {}".format(round(f1_avg/len(dataloader),3)))
    print("Accuracy {}".format(round(acc/len(dataloader),3)))
    print("Accuracy YN{}".format(round(acc_yn/c_yn,3)))


def compute_f1(gold_toks, pred_toks):
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

In [11]:
eval_gpt_open_ended(model, test_loader)

Testing:   0%|          | 0/225 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram or

Question:  ['is this an axial plane']
Answer:  ['yes']
Generated_Answer:  yes


Testing:   1%|          | 2/225 [00:00<01:00,  3.68it/s]

Question:  ['are 12 ribs present in the image']
Answer:  ['yes']
Generated_Answer:  yes


Testing:   2%|▏         | 4/225 [00:00<00:49,  4.44it/s]

...

Question:  ['this image is taken in what plane']
Answer:  ['axial']
Generated_Answer:  axial
Question:  ['are the ventricles an abnormal size']
Answer:  ['no']
Generated_Answer:  yes
Question:  ['is the size of the ventricles abnormal']
Answer:  ['no']
Generated_Answer:  no


Testing:  99%|█████████▉| 223/225 [00:32<00:00,  9.99it/s]

Question:  ['this image is consistent with what condition']
Answer:  ['appendicitis']
Generated_Answer:  ct
Question:  ['what condition does the patient have']
Answer:  ['appendicitis']
Generated_Answer:  ct
Question:  ['where is there obstruction present']
Answer:  ['proximal aspect of the appendix']
Generated_Answer:  right


Testing: 100%|██████████| 225/225 [00:32<00:00,  6.94it/s]

Question:  ['is the gi tract is highlighted by contrast']
Answer:  ['yes']
Generated_Answer:  yes
Question:  ['do you see cardiomegaly']
Answer:  ['no']
Generated_Answer:  no
BLEU 0.267
F1 0.267
Accuracy 0.311
Accuracy YN0.455
